In [1]:
%pip install sentence-transformers faiss-cpu pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install faiss-cpu==1.7.4

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.12.0, 1.13.0)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import os
from tqdm import tqdm

In [27]:
import pickle

In [4]:
df = pd.read_csv('../Data_cleaning/Articles_with_parts.csv')
df.head()

,url,Part,title,content
0,https://indiankanoon.org/doc/367586/,Part III,Article 14 in Constitution of India\n ...,14. Equality before law The State shall not de...
1,https://indiankanoon.org/doc/1456610/,Part I,Article 2 in Constitution of India\n ...,2. Admission or establishment of new States Pa...
2,https://indiankanoon.org/doc/1659104/,Part I,Article 3 in Constitution of India\n ...,3. Formation of new States and alteration of a...
3,https://indiankanoon.org/doc/1015123/,Part I,Article 4 in Constitution of India\n ...,4. Laws made under articles 2 and 3 to provide...
4,https://indiankanoon.org/doc/1937835/,Part II,Article 5 in Constitution of India\n ...,5. Citizenship at the commencement of the Cons...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      194 non-null    object
 1   Part     194 non-null    object
 2   title    194 non-null    object
 3   content  194 non-null    object
dtypes: object(4)
memory usage: 6.2+ KB


In [ ]:
# If the column name is different, update here:
# the content column is converted to a list.
texts = df["content"].tolist()
len(texts)

194

In [20]:
# ============================================================
# 4️⃣ CHUNKING FUNCTION
#    Break large articles into smaller 300–500 token chunks
# ============================================================

def split_into_chunks(text, max_length=400):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_length):
        chunk = " ".join(words[i:i+max_length])
        chunks.append(chunk)
    return chunks

In [22]:
# ============================================================
# 5️⃣ CREATE FINAL LIST OF CHUNKS
# ============================================================

all_chunks = []

for article in texts:
    chunks = split_into_chunks(article)
    all_chunks.extend(chunks)

print("Total chunks:", len(all_chunks))

Total chunks: 251


In [23]:
# 6️⃣ LOAD SENTENCE TRANSFORMER MODEL
model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
# ============================================================
# 7️⃣ ENCODE CHUNKS INTO EMBEDDINGS
# ============================================================

embeddings = model.encode(all_chunks, show_progress_bar=True)
# conversion of embedding in array
embeddings = np.array(embeddings).astype("float32")
embeddings.shape

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

(251, 384)

In [25]:
# ============================================================
# 8️⃣ CREATE FAISS INDEX
# ============================================================

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [26]:
# ============================================================
# 9️⃣ CREATE FOLDER TO SAVE
# ============================================================

os.makedirs("vector_store", exist_ok=True)

In [28]:
# ============================================================
# 🔟 SAVE FAISS + CHUNKS
# ============================================================

faiss.write_index(index, "vector_store/faiss_articles.bin")

with open("vector_store/chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

print("All files saved successfully!")

All files saved successfully!
